In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
emissions_data = pd.read_csv("../data/raw/emissions_owid.csv")

In [3]:
emissions_data.dtypes

country                         object
year                             int64
iso_code                        object
population                     float64
gdp                            float64
                                ...   
temperature_change_from_n2o    float64
total_ghg                      float64
total_ghg_excluding_lucf       float64
trade_co2                      float64
trade_co2_share                float64
Length: 79, dtype: object

In [4]:
emissions_data_cleaned = emissions_data[["country", "year", "iso_code", "population", "gdp", "co2_per_capita", "co2_per_gdp", "coal_co2_per_capita",
                                         "energy_per_capita", "energy_per_gdp", "gas_co2_per_capita", "land_use_change_co2_per_capita", "oil_co2_per_capita"]]

In [5]:
emissions_data_cleaned = emissions_data_cleaned.rename(columns={"iso_code" : "country_code"})

In [6]:
emissions_data_cleaned["country_code"].isna().sum()

7929

In [7]:
emissions_data_cleaned = emissions_data_cleaned[emissions_data_cleaned["country_code"].notna()]

In [8]:
emissions_data_cleaned["land_use_change_co2_per_capita_lag"] = emissions_data_cleaned.groupby("country")["land_use_change_co2_per_capita"].shift(1)

In [9]:
emissions_data_cleaned[['gdp', 'co2_per_capita', 'co2_per_gdp']].isna().sum()

gdp               27032
co2_per_capita    19491
co2_per_gdp       27590
dtype: int64

In [10]:
emissions_data_cleaned['population'] = emissions_data_cleaned.groupby('country')['population'].transform(lambda x: x.ffill().bfill())

In [11]:
emissions_data_cleaned = emissions_data_cleaned[emissions_data_cleaned["population"].notna()]

In [12]:
emissions_data_cleaned = emissions_data_cleaned.copy()
emissions_data_cleaned["population"] = emissions_data_cleaned["population"].astype(int)

In [13]:
# Carbon Tax is more relevant past 1990:
emissions_data_cleaned = emissions_data_cleaned[(emissions_data_cleaned["year"] >= 1990) & (emissions_data_cleaned["year"] <= 2022)]

In [14]:
emissions_data_cleaned[emissions_data_cleaned["gdp"].isna()]["country"].unique()

array(['Andorra', 'Anguilla', 'Antigua and Barbuda', 'Aruba', 'Bahamas',
       'Belize', 'Bermuda', 'Bhutan', 'Bonaire Sint Eustatius and Saba',
       'British Virgin Islands', 'Brunei', 'Cook Islands', 'Curacao',
       'East Timor', 'Eritrea', 'Faroe Islands', 'Fiji',
       'French Polynesia', 'Greenland', 'Grenada', 'Guyana', 'Kiribati',
       'Liechtenstein', 'Macao', 'Maldives', 'Marshall Islands',
       'Micronesia (country)', 'Monaco', 'Montserrat', 'Nauru',
       'New Caledonia', 'Niue', 'Palau', 'Papua New Guinea',
       'Saint Helena', 'Saint Kitts and Nevis',
       'Saint Pierre and Miquelon', 'Saint Vincent and the Grenadines',
       'Samoa', 'San Marino', 'Sint Maarten (Dutch part)',
       'Solomon Islands', 'Somalia', 'South Sudan', 'Sudan', 'Suriname',
       'Tonga', 'Turks and Caicos Islands', 'Tuvalu',
       'United Arab Emirates', 'Vanuatu', 'Vatican', 'Wallis and Futuna'],
      dtype=object)

In [15]:
#interpolate 1991 and 1992 ARE gdp from WB Data
uae_scaling1990 = 40641368064 / 165048348648.546
uae_scaling1993 = 55149662208 / 188567499474.299
uae_scaling1994 = 64654417920 / 205875343585.347
uae_scaling1995 = 75243839488 / 224249304379.684
uae_scaling1996 = 87750959104 / 241595714092.327
years = [1990, 1993, 1994, 1995, 1996]
scaling_factors = [uae_scaling1990, uae_scaling1993, uae_scaling1994, uae_scaling1995, uae_scaling1996]
equation = np.polyfit(years, scaling_factors, 1)
slope = equation[0]
intercept = equation[1]
uae_scaling1991 = slope * 1991 + intercept
uae_scaling1992 = slope * 1992 + intercept

In [16]:
WB_uae_1991 = 172097530003.326
WB_uae_1992 = 181907133314.7
owid_uae_1991 = WB_uae_1991 * uae_scaling1991
owid_uae_1992 = WB_uae_1992 * uae_scaling1992

In [17]:
emissions_data_cleaned = emissions_data_cleaned.copy()

In [18]:
emissions_data_cleaned = emissions_data_cleaned.copy()
emissions_data_cleaned.loc[
    (emissions_data_cleaned["country_code"] == "ARE") &
    (emissions_data_cleaned["year"] == 1991), "gdp"
] = owid_uae_1991

emissions_data_cleaned.loc[
    (emissions_data_cleaned["country_code"] == "ARE") &
    (emissions_data_cleaned["year"] == 1992), "gdp"
] = owid_uae_1992


In [19]:
emissions_data_cleaned = emissions_data_cleaned[emissions_data_cleaned["gdp"].notna()]

In [20]:
emissions_data_cleaned = emissions_data_cleaned[emissions_data_cleaned["co2_per_capita"].notna()]

In [21]:
emissions_data_cleaned["gdp"] = pd.to_numeric(emissions_data_cleaned["gdp"], errors='coerce')

In [22]:
emissions_data_cleaned[[
    "coal_co2_per_capita", "energy_per_capita", "energy_per_gdp", "gas_co2_per_capita", 
    "land_use_change_co2_per_capita_lag", "oil_co2_per_capita"]].isna().sum()

coal_co2_per_capita                   1293
energy_per_capita                      136
energy_per_gdp                         138
gas_co2_per_capita                    1719
land_use_change_co2_per_capita_lag      66
oil_co2_per_capita                       0
dtype: int64

In [23]:
missing_coal = emissions_data_cleaned[emissions_data_cleaned['coal_co2_per_capita'].isna()]
missing_by_country = missing_coal.groupby('country').size()

In [24]:
# Examining data and external energy reports and emission profiles, these countries have minimal coal usage.
# Empty coal emission cells can be filled with 0. 
emissions_data_cleaned["coal_co2_per_capita"].fillna(0, inplace=True)

In [25]:
emissions_data_cleaned["land_use_change_co2_per_capita_lag"].fillna(0, inplace=True)
emissions_data_cleaned["land_use_change_co2_per_capita"].fillna(0, inplace=True)

In [26]:
pd.set_option('display.max_rows', None)
missing_energy = emissions_data_cleaned[emissions_data_cleaned['energy_per_capita'].isna()]
missing_energy.groupby('country').size()

country
Afghanistan                      1
Albania                          1
Angola                           1
Armenia                          3
Bahrain                          1
Barbados                         1
Benin                            1
Bolivia                          1
Bosnia and Herzegovina           3
Botswana                         1
Burkina Faso                     1
Burundi                          1
Cambodia                         1
Cameroon                         1
Cape Verde                       1
Central African Republic         1
Chad                             1
Comoros                          1
Congo                            1
Costa Rica                       1
Cote d'Ivoire                    1
Cuba                             1
Democratic Republic of Congo     1
Djibouti                         1
Dominica                         1
Dominican Republic               1
El Salvador                      1
Equatorial Guinea                1
Eswatini    

In [27]:
pd.reset_option('display.max_rows')

In [28]:
emissions_data_cleaned = emissions_data_cleaned[~(
    ((emissions_data_cleaned["country"].isin(["Montenegro", "Serbia"])) &
     (emissions_data_cleaned["year"] < 2006))
)]

In [29]:
emissions_data_cleaned.loc[:, "energy_per_capita"] = (
    emissions_data_cleaned.groupby("country")["energy_per_capita"]
    .transform(lambda x: x.interpolate())
)

emissions_data_cleaned.loc[:, "energy_per_gdp"] = (
    emissions_data_cleaned.groupby("country")["energy_per_gdp"]
    .transform(lambda x: x.interpolate())
)


In [30]:
emissions_data_cleaned = emissions_data_cleaned.copy()

In [31]:
emissions_data_cleaned["gas_co2_per_capita"] = emissions_data_cleaned["gas_co2_per_capita"].fillna(0)

In [32]:
emissions_data_cleaned["co2_per_capita_minus_5"] = emissions_data_cleaned.groupby('country')["co2_per_capita"].shift(5)
emissions_data_cleaned["co2_per_capita_plus_5"] = emissions_data_cleaned.groupby('country')["co2_per_capita"].shift(-5)

emissions_data_cleaned["co2_per_gdp_minus_5"] = emissions_data_cleaned.groupby('country')["co2_per_gdp"].shift(5)
emissions_data_cleaned["co2_per_gdp_plus_5"] = emissions_data_cleaned.groupby('country')["co2_per_gdp"].shift(-5)

In [33]:
emissions_data_cleaned["co2_per_capita_prior_trend"] = (emissions_data_cleaned["co2_per_capita"] - emissions_data_cleaned["co2_per_capita_minus_5"]) / 5
emissions_data_cleaned["co2_per_capita_future_trend"] = (emissions_data_cleaned["co2_per_capita_plus_5"] - emissions_data_cleaned["co2_per_capita"]) / 5

emissions_data_cleaned["co2_per_gdp_prior_trend"] = (emissions_data_cleaned["co2_per_gdp"] - emissions_data_cleaned["co2_per_gdp_minus_5"]) / 5
emissions_data_cleaned["co2_per_gdp_future_trend"] = (emissions_data_cleaned["co2_per_gdp_plus_5"] - emissions_data_cleaned["co2_per_gdp"]) / 5

In [34]:
emissions_data_cleaned.to_csv("../data/cleaned/emissions_owid_cleaned.csv", index=False, float_format='%.6f')

In [35]:
general_data = pd.read_csv("../data/raw/carbon_tax_wb_general.csv", header = 1)
price_data = pd.read_csv("../data/raw/carbon_tax_wb_prices.csv", header = 1)

In [36]:
general_data.columns

Index(['Unique ID', 'Instrument name', 'Type', 'Status',
       'Jurisdiction covered', 'Share of jurisdiction emissions covered',
       'Price on 1 April', '2020', '2021', '2022', '2023', '2024', '2025',
       'Change', 'Government revenue', '2019', '2020.1', '2021.1', '2022.1',
       '2023.1', '2024.1', 'Change.1', 'Gases covered', 'Electricity and heat',
       'Industry', 'Mining and extractives', 'Transport', 'Aviation',
       'Buildings', 'Agriculture, forestry and fishing fuel use',
       'Agricultural emissions', 'Waste', 'LULUCF', 'Fuels covered',
       'Allocation approaches', 'Price or market management',
       'Point of Regulation', 'Offset eligibility', 'Description',
       'Recent developments', 'Coverage', 'Pricing and allocation',
       'Compliance', 'Relation to other instruments'],
      dtype='object')

In [37]:
general_data_cleaned = general_data[["Unique ID", "Type", "Status", "Jurisdiction covered"]]

In [38]:
general_data_cleaned = general_data_cleaned.rename(columns={"Jurisdiction covered" : "country", "Status" : "status", 
                                                            "Unique ID" : "ID", "Type" : "type"})

In [39]:
general_data_cleaned

,ID,type,status,country
0,Tax_AL,Carbon tax,Implemented,Albania
1,ETS_AL,ETS,Under consideration,Albania
2,Tax_CA_Alberta,Carbon tax,Abolished,Alberta
3,ETS_CA_Alberta,ETS,Implemented,Alberta
4,Tax_AN,Carbon tax,Implemented,Andorra
...,...,...,...,...
126,Tax_UY,Carbon tax,Implemented,Uruguay
127,ETS_VN,ETS,Under development,Viet Nam
128,ETS_US_Washington,ETS,Implemented,Washington
129,Tax_MX_Yucatan,Carbon tax,Implemented,Yucatan


In [40]:
general_data_cleaned = general_data_cleaned[(general_data_cleaned["status"] == "Implemented") | (general_data_cleaned["status"] == "Abolished")]

In [41]:
valid_countries = set(emissions_data_cleaned["country"].unique())
valid_countries.add("EU")

In [42]:
general_data_cleaned = general_data_cleaned[general_data_cleaned["country"].isin(valid_countries)]

In [43]:
price_data = price_data.rename(columns={"Unique ID" : "ID"})
price_data.columns

Index(['ID', 'Name of the initiative', 'Instrument Type', 'Region',
       'Income group', 'Metric', '1990', '1991', '1992', '1993', '1994',
       '1995', '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003',
       '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012',
       '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021',
       '2022', '2023', '2024', '2025'],
      dtype='object')

In [44]:
price_data_cleaned = price_data.drop(["Name of the initiative", "Instrument Type", "Region", "Income group", "Metric"], axis="columns")

In [45]:
price_data_cleaned = price_data_cleaned.fillna(0) #No tax in that year

In [46]:
general_data_cleaned["ID"] = general_data_cleaned["ID"].astype(str)
price_data_cleaned["ID"] = price_data_cleaned["ID"].astype(str)

In [47]:
carbon_tax_combined_cleaned = pd.merge( 
    general_data_cleaned, price_data_cleaned,
    on="ID", how="left")

In [48]:
carbon_tax_combined_cleaned.to_csv("../data/cleaned/carbon_tax_wb_cleaned.csv", index=False, float_format='%.6f')

In [49]:
eu_countries = [
    "Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czech Republic",
    "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary",
    "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta",
    "Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia",
    "Spain", "Sweden"
]

In [50]:
eu_rows = carbon_tax_combined_cleaned[carbon_tax_combined_cleaned['country'] == 'EU']
non_eu_rows = carbon_tax_combined_cleaned[carbon_tax_combined_cleaned['country'] != 'EU']

In [51]:
expanded_eu_rows = pd.concat([
    eu_rows.assign(country=country) for country in eu_countries], ignore_index=True)

In [52]:
expanded_eu_rows

,ID,type,status,country,1990,1991,1992,1993,1994,1995,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,ETS_EU,ETS,Implemented,Austria,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
1,ETS_EU,ETS,Implemented,Belgium,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
2,ETS_EU,ETS,Implemented,Bulgaria,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
3,ETS_EU,ETS,Implemented,Croatia,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
4,ETS_EU,ETS,Implemented,Cyprus,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
5,ETS_EU,ETS,Implemented,Czech Republic,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
6,ETS_EU,ETS,Implemented,Denmark,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
7,ETS_EU,ETS,Implemented,Estonia,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
8,ETS_EU,ETS,Implemented,Finland,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124
9,ETS_EU,ETS,Implemented,France,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.26372,24.505716,18.53652,49.779548,86.526108,96.298125,61.301547,70.370124


In [53]:
carbon_tax_combined_cleaned = pd.concat([non_eu_rows, expanded_eu_rows], ignore_index=True)

In [54]:
carbon_tax_combined_cleaned

,ID,type,status,country,1990,1991,1992,1993,1994,1995,...,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Tax_AL,Carbon tax,Implemented,Albania,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,13.111355,13.691905
1,Tax_AR,Carbon tax,Implemented,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,8.914348,6.187569,6.342015,5.321599,4.995626,3.218257,0.811554,5.327915
2,ETS_AU1,ETS,Abolished,Australia,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,ETS_AU2,ETS,Implemented,Australia,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,21.903750,21.816120
4,ETS_AT,ETS,Implemented,Austria,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,35.343750,48.370500,48.546000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,ETS_EU,ETS,Implemented,Romania,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.263720,24.505716,18.536520,49.779548,86.526108,96.298125,61.301547,70.370124
66,ETS_EU,ETS,Implemented,Slovakia,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.263720,24.505716,18.536520,49.779548,86.526108,96.298125,61.301547,70.370124
67,ETS_EU,ETS,Implemented,Slovenia,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.263720,24.505716,18.536520,49.779548,86.526108,96.298125,61.301547,70.370124
68,ETS_EU,ETS,Implemented,Spain,0.0,0.0,0.0,0.0,0.0,0.0,...,5.921776,5.644848,16.263720,24.505716,18.536520,49.779548,86.526108,96.298125,61.301547,70.370124


In [55]:
carbon_tax_long = carbon_tax_combined_cleaned.melt(
    id_vars = ["country", "type"],
    var_name = "year",
    value_name = "tax_price")

In [56]:
pd.set_option('display.max_rows', None)

In [57]:
carbon_tax_long = carbon_tax_long[(carbon_tax_long["year"] != "ID") & (carbon_tax_long["year"] != "status")]

In [58]:
carbon_tax_long = carbon_tax_long.copy()

In [59]:
carbon_tax_long["year"] = carbon_tax_long["year"].astype(int)
carbon_tax_long = carbon_tax_long[pd.to_numeric(carbon_tax_long["tax_price"], errors="coerce").notnull()]
carbon_tax_long = carbon_tax_long[carbon_tax_long["tax_price"] > 0]
carbon_tax_long["tax_price"] = carbon_tax_long["tax_price"].astype(float)

In [60]:
def get_tax_type(types):
    types_set = set(types)
    if types_set == {"Carbon tax"}:
        return 1
    elif types_set == {"ETS"}:
        return 2
    elif types_set == {"Carbon tax", "ETS"}:
        return 3
    else:
        return 0

In [61]:
carbon_tax_long_cleaned = carbon_tax_long.groupby(["country", "year"]).agg(
    tax_price = ("tax_price", "max"),
    tax_type = ("type", get_tax_type)).reset_index()

In [62]:
carbon_tax_long_cleaned.to_csv("../data/cleaned/carbon_tax_wb_yearly_cleaned.csv", index=False, float_format='%.6f')

In [63]:
combined_data = emissions_data_cleaned.merge(carbon_tax_long_cleaned, on=["country", "year"], how="left")

In [64]:
combined_data["tax_type"] = combined_data["tax_type"].fillna(0).astype(int)
combined_data["tax_price"] = combined_data["tax_price"].fillna(0)

In [65]:
def compute_years_since_tax(group):
    active = group["tax_type"] > 0
    if active.any():
        first_year = group.loc[active, "year"].min()
        group["years_since_tax"] = group["year"] - first_year
        group.loc[group["year"] < first_year, "years_since_tax"] = -1
    else:
        group["years_since_tax"] = -1
    return group

In [66]:
combined_data = combined_data.groupby("country").apply(compute_years_since_tax)

In [67]:
combined_data.to_csv("../data/cleaned/combined_data.csv", index=False, float_format="%.6f")

In [68]:
energy_data = pd.read_csv("../data/raw/energy_owid.csv")

In [69]:
energy_data_cleaned = energy_data[["country", "year", "iso_code", "coal_share_energy", "gas_share_energy", "nuclear_share_energy",
                                   "oil_share_energy", "renewables_share_energy"]]

In [70]:
energy_data_cleaned = energy_data_cleaned.rename(columns={"iso_code" : "country_code"})

In [71]:
energy_data_cleaned = energy_data_cleaned[energy_data_cleaned["country_code"].notna()]

In [72]:
energy_data_cleaned = energy_data_cleaned[(energy_data_cleaned["year"] >= 1990) & (energy_data_cleaned["year"] <= 2022)]

In [73]:
energy_data_cleaned[['coal_share_energy', 'nuclear_share_energy', 'oil_share_energy', "gas_share_energy", "renewables_share_energy"]].isna().sum()

coal_share_energy          4928
nuclear_share_energy       4928
oil_share_energy           4928
gas_share_energy           4928
renewables_share_energy    4928
dtype: int64

In [74]:
energy_data_cleaned.to_csv("../data/cleaned/energy_data_cleaned.csv", index=False, float_format="%.6f")

In [75]:
energy_data_cleaned["energy_mix_filled"] = energy_data_cleaned[['coal_share_energy', 'nuclear_share_energy', 'oil_share_energy', "gas_share_energy", "renewables_share_energy"]].isna().any(axis=1)
energy_data["energy_mix_filled"] = energy_data_cleaned["energy_mix_filled"].astype(int)

In [76]:
energy_data_new = pd.read_csv("../data/raw/energy_wb.csv")

In [77]:
import re
energy_data_new.columns = [
    int(re.search(r"\d{4}", col).group()) if re.search(r"\d{4}", col) else col
    for col in energy_data_new.columns
]

In [78]:
energy_data_new.columns

Index(['Country Name', 'Country Code',  'Series Name',  'Series Code',
                 1990,           1991,           1992,           1993,
                 1994,           1995,           1996,           1997,
                 1998,           1999,           2000,           2001,
                 2002,           2003,           2004,           2005,
                 2006,           2007,           2008,           2009,
                 2010,           2011,           2012,           2013,
                 2014,           2015,           2016,           2017,
                 2018,           2019,           2020,           2021,
                 2022,           2023,           2024],
      dtype='object')

In [79]:
energy_data_cleaned = energy_data_new.drop("Series Code", axis = 1)

In [80]:
energy_data_cleaned.columns

Index(['Country Name', 'Country Code',  'Series Name',           1990,
                 1991,           1992,           1993,           1994,
                 1995,           1996,           1997,           1998,
                 1999,           2000,           2001,           2002,
                 2003,           2004,           2005,           2006,
                 2007,           2008,           2009,           2010,
                 2011,           2012,           2013,           2014,
                 2015,           2016,           2017,           2018,
                 2019,           2020,           2021,           2022,
                 2023,           2024],
      dtype='object')

In [81]:
energy_data_cleaned = energy_data_cleaned[energy_data_cleaned["Country Code"].notna()]
code_to_country = combined_data.drop_duplicates("country_code")[["country_code", "country"]].set_index("country_code")["country"]
energy_data_cleaned["Country Name"] = energy_data_cleaned["Country Code"].map(code_to_country)

In [82]:
code_to_country

country_code
AFG                     Afghanistan
ALB                         Albania
DZA                         Algeria
AGO                          Angola
ARG                       Argentina
ARM                         Armenia
AUS                       Australia
AUT                         Austria
AZE                      Azerbaijan
BHR                         Bahrain
BGD                      Bangladesh
BRB                        Barbados
BLR                         Belarus
BEL                         Belgium
BEN                           Benin
BOL                         Bolivia
BIH          Bosnia and Herzegovina
BWA                        Botswana
BRA                          Brazil
BGR                        Bulgaria
BFA                    Burkina Faso
BDI                         Burundi
KHM                        Cambodia
CMR                        Cameroon
CAN                          Canada
CPV                      Cape Verde
CAF        Central African Republic
TCD            

In [83]:
energy_data_cleaned = energy_data_cleaned.melt(id_vars=["Country Name", "Series Name"], var_name="Year", value_name="Value")

In [84]:
energy_data_cleaned = energy_data_cleaned[energy_data_cleaned["Year"]!= "Country Code"]

In [85]:
energy_data_cleaned = energy_data_cleaned.copy()
energy_data_cleaned["Value"] = pd.to_numeric(energy_data_cleaned["Value"], errors="coerce")

In [86]:
energy_data_cleaned = energy_data_cleaned.pivot_table(index=["Country Name", "Year"], columns="Series Name", values="Value")

In [87]:
energy_data_cleaned.columns.name = None

In [88]:
energy_data_cleaned = energy_data_cleaned.reset_index()

In [89]:
energy_data_cleaned["Country Name"] = energy_data_cleaned["Country Name"].ffill()

In [90]:
energy_data_cleaned = energy_data_cleaned.rename(columns={"Country Name" : "country", "Year" : "year",
                                                          "Alternative and nuclear energy (% of total energy use)" : "nuclear_pct",
                                                         "Fossil fuel energy consumption (% of total)" : "fossil_pct",
                                                         "Renewable energy consumption (% of total final energy consumption)" : "renewable_pct"})

In [91]:
energy_data_cleaned = energy_data_cleaned[energy_data_cleaned["country"].isin(valid_countries)]

In [100]:
energy_cols = ["nuclear_pct", "fossil_pct", "renewable_pct"]

In [92]:
for col in ["nuclear_pct", "fossil_pct", "renewable_pct"]:
    energy_data_cleaned[col + "_filled"] = energy_data_cleaned[col].isna().astype(int)

In [93]:
#All 9 countries yet to implement nuclear power, Burundi, Guinea, and Mali have initiated discussions or agreements regarding nuclear energy.
non_nuclear = [
    'Afghanistan', 'Barbados', 'Burundi', 'Central African Republic',
    'Comoros', 'Djibouti', 'Dominica', 'Guinea', 'Guinea-Bissau',
    'Lesotho', 'Liberia', 'Malawi', 'Mali', 'Mauritania',
    'Sao Tome and Principe', 'Seychelles', 'Sierra Leone', 'Saint Lucia',
    'Cape Verde', 'Gambia', 'Palestine'
]

energy_data_cleaned.loc[
    (energy_data_cleaned['country'].isin(non_nuclear)) &
    (energy_data_cleaned['nuclear_pct'].isna()),
    'nuclear_pct'
] = 0

In [94]:
energy_data_cleaned.loc[
    energy_data_cleaned['country'].isin(non_nuclear), 
    'fossil_pct'
] = 100 - (
    energy_data_cleaned['renewable_pct'] + 
    energy_data_cleaned['nuclear_pct']
)

In [95]:
for col in ["nuclear_pct", "fossil_pct", "renewable_pct"]:
    energy_data_cleaned[col] = energy_data_cleaned.groupby("country")[col].transform(lambda x: x.interpolate(method="linear").ffill().bfill())

In [96]:
for col in ["nuclear_pct", "fossil_pct"]:
    energy_data_cleaned[col] = energy_data_cleaned.groupby("country")[col].transform(lambda x: x.ffill().bfill())

In [97]:
energy_data_cleaned[['nuclear_pct', 'fossil_pct', 'renewable_pct']].isna().sum()

nuclear_pct      0
fossil_pct       0
renewable_pct    0
dtype: int64

In [98]:
energy_data_cleaned

,country,year,nuclear_pct,fossil_pct,renewable_pct,nuclear_pct_filled,fossil_pct_filled,renewable_pct_filled
0,Afghanistan,1990,0.00,77.00,23.0,1,1,0
1,Afghanistan,1991,0.00,76.30,23.7,1,1,0
2,Afghanistan,1992,0.00,72.60,27.4,1,1,0
3,Afghanistan,1993,0.00,71.50,28.5,1,1,0
4,Afghanistan,1994,0.00,69.90,30.1,1,1,0
5,Afghanistan,1995,0.00,68.20,31.8,1,1,0
6,Afghanistan,1996,0.00,66.10,33.9,1,1,0
7,Afghanistan,1997,0.00,63.60,36.4,1,1,0
8,Afghanistan,1998,0.00,62.00,38.0,1,1,0
9,Afghanistan,1999,0.00,56.60,43.4,1,1,0


In [102]:
combined_data = combined_data.reset_index(drop=True)
energy_data_cleaned = energy_data_cleaned.reset_index(drop=True)


combined_data = combined_data.merge(energy_data_cleaned, on=["country", "year"], how="left")

In [103]:
print(combined_data.index.names)
print(combined_data.columns)

[None]
Index(['country', 'year', 'country_code', 'population', 'gdp',
       'co2_per_capita', 'co2_per_gdp', 'coal_co2_per_capita',
       'energy_per_capita', 'energy_per_gdp', 'gas_co2_per_capita',
       'land_use_change_co2_per_capita', 'oil_co2_per_capita',
       'land_use_change_co2_per_capita_lag', 'co2_per_capita_minus_5',
       'co2_per_capita_plus_5', 'co2_per_gdp_minus_5', 'co2_per_gdp_plus_5',
       'co2_per_capita_prior_trend', 'co2_per_capita_future_trend',
       'co2_per_gdp_prior_trend', 'co2_per_gdp_future_trend', 'tax_price',
       'tax_type', 'years_since_tax', 'nuclear_pct', 'fossil_pct',
       'renewable_pct', 'nuclear_pct_filled', 'fossil_pct_filled',
       'renewable_pct_filled'],
      dtype='object')


In [105]:
combined_data = combined_data[combined_data["nuclear_pct"].notna()]

In [106]:
combined_data.to_csv("../data/cleaned/combined_data.csv", index=False, float_format="%.6f")